In [34]:
import torch
import torch.nn as nn
from sklearn.metrics import f1_score, precision_score, recall_score
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

In [35]:
class MyDataset(Dataset):
    def __init__(self, file_path):
        self.data = pd.read_csv(file_path)
        self.x = np.array(self.data.iloc[:, :-1])
        self.y = np.array(self.data.iloc[:, -1])
        
    def __getitem__(self, index):
        return torch.tensor(self.x[index], dtype=torch.float32), torch.tensor(self.y[index], dtype=torch.long)
        
    def __len__(self):
        return len(self.data)


In [36]:
model = MLP()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [37]:
# L2 Regülerizasyonu (Weight Decay):


In [38]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(13, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.sigmoid(self.fc3(x))
        return x
    
# L2 Regülerizasyonu
l2_reg = 0.01
for param in model.parameters():
    param.data -= l2_reg * param.data.norm(2)

In [39]:
train_dataset = MyDataset('cure_the_princess_train.csv')
val_dataset = MyDataset('cure_the_princess_validation.csv')
test_dataset = MyDataset('cure_the_princess_test.csv')


In [40]:
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [41]:
def train(model, train_dataloader, optimizer, loss_fn):
    model.train()
    for batch, (data, target) in enumerate(train_dataloader):
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()


In [42]:
def evaluate(model, val_dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in val_dataloader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return 100 * correct / total


In [47]:
# Test verisi üzerinde modelin doğruluğunu hesaplayalım
def evaluate_L2(model, dataloader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data, target in dataloader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            y_true += target.numpy().tolist()
            y_pred += predicted.numpy().tolist()
    return (100 * np.sum(np.array(y_true) == np.array(y_pred)) / len(y_true),
            precision_score(y_true, y_pred),
            recall_score(y_true, y_pred),
            f1_score(y_true, y_pred))


In [51]:
for epoch in range(10):
    train(model, train_dataloader, optimizer, loss_fn)
    acc = evaluate(model, val_dataloader)
    print('Epoch:', epoch+1, '| Validation Accuracy:', acc)

Epoch: 1 | Validation Accuracy: 85.98726114649682
Epoch: 2 | Validation Accuracy: 86.62420382165605
Epoch: 3 | Validation Accuracy: 87.89808917197452
Epoch: 4 | Validation Accuracy: 85.6687898089172
Epoch: 5 | Validation Accuracy: 85.98726114649682
Epoch: 6 | Validation Accuracy: 87.57961783439491
Epoch: 7 | Validation Accuracy: 88.85350318471338
Epoch: 8 | Validation Accuracy: 88.21656050955414
Epoch: 9 | Validation Accuracy: 87.89808917197452
Epoch: 10 | Validation Accuracy: 89.80891719745223


In [52]:
test_acc = evaluate(model, test_dataloader)
print('Test Accuracy:', test_acc)

Test Accuracy: 84.4559585492228


In [55]:
# Early stopping ile en iyi modeli seçme
best_val_acc = 0
patience = 3
counter = 0
for epoch in range(10):
    train(model, train_dataloader, optimizer, loss_fn)
    val_acc = evaluate_L2(model, val_dataloader)[0]
    print('Epoch:', epoch+1, '| Validation Accuracy:', val_acc)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model_L2.pt')
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print('Early stopping.')
            break

# En iyi modeli yükleyelim
model.load_state_dict(torch.load('best_model_L2.pt'))


Epoch: 1 | Validation Accuracy: 92.35668789808918
Epoch: 2 | Validation Accuracy: 92.67515923566879
Epoch: 3 | Validation Accuracy: 92.67515923566879
Epoch: 4 | Validation Accuracy: 91.71974522292993
Epoch: 5 | Validation Accuracy: 92.03821656050955
Early stopping.


<All keys matched successfully>

In [57]:
# Test verisi üzerinde modelin doğruluğunu, precision, recall ve f1_score değerlerini hesaplayalım
test_acc, test_precision, test_recall, test_f1 = evaluate_L2(model, test_dataloader)
print('Test Accuracy:', test_acc)
print('Test Precision:', test_precision)
print('Test Recall:', test_recall)
print('Test F1-score:', test_f1)

Test Accuracy: 87.56476683937824
Test Precision: 0.8578431372549019
Test Recall: 0.9020618556701031
Test F1-score: 0.879396984924623


In [58]:
# Dropout Regülerizasyonu

In [59]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(13, 100)
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(100, 50)
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout1(x)
        x = nn.functional.relu(self.fc2(x))
        x = self.dropout2(x)
        x = nn.functional.sigmoid(self.fc3(x))
        return x

In [61]:
# Model, Loss ve Optimizer
model_dropout = MLP()
loss_fn = nn.CrossEntropyLoss()
optimizer_dropout = optim.SGD(model_dropout.parameters(), lr=0.01)


In [62]:
# Eğitim fonksiyonu
def train(model, train_dataloader, optimizer, loss_fn):
    model.train()
    for batch, (data, target) in enumerate(train_dataloader):
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()


In [63]:
# Doğrulama fonksiyonu
def evaluate(model, dataloader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data, target in dataloader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            y_true += target.numpy().tolist()
            y_pred += predicted.numpy().tolist()
    accuracy = (np.array(y_true) == np.array(y_pred)).mean() * 100
    f1 = f1_score(y_true, y_pred, average='weighted') * 100
    precision = precision_score(y_true, y_pred, average='weighted') * 100
    recall = recall_score(y_true, y_pred, average='weighted') * 100
    return accuracy, f1, precision, recall


In [64]:
# Early stopping ile en iyi modeli seçme
best_val_acc = 0
patience = 3
counter = 0
for epoch in range(10):
    train(model_dropout, train_dataloader, optimizer_dropout, loss_fn)
    val_acc, val_f1, val_precision, val_recall = evaluate(model_dropout, val_dataloader)
    print('Epoch:', epoch+1, '| Validation Accuracy:', val_acc, '| Validation F1:', val_f1, '| Validation Precision:', val_precision, '| Validation Recall:', val_recall)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model_dropout.state_dict(), 'best_model_dropout.pt')
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print('Early stopping.')
            break

# En iyi modeli yükleyelim
model_dropout.load_state_dict(torch.load('best_model_dropout.pt'))


Epoch: 1 | Validation Accuracy: 58.28025477707006 | Validation F1: 54.90371828383296 | Validation Precision: 62.12323085323293 | Validation Recall: 58.28025477707006
Epoch: 2 | Validation Accuracy: 53.82165605095541 | Validation F1: 43.50534586935888 | Validation Precision: 63.00431765862423 | Validation Recall: 53.82165605095541
Epoch: 3 | Validation Accuracy: 73.24840764331209 | Validation F1: 73.23537927041112 | Validation Precision: 73.31759101030501 | Validation Recall: 73.24840764331209
Epoch: 4 | Validation Accuracy: 60.509554140127385 | Validation F1: 56.334728148468805 | Validation Precision: 66.61617934700837 | Validation Recall: 60.509554140127385
Epoch: 5 | Validation Accuracy: 71.01910828025477 | Validation F1: 70.13516545907412 | Validation Precision: 73.66516211074709 | Validation Recall: 71.01910828025477
Epoch: 6 | Validation Accuracy: 73.24840764331209 | Validation F1: 72.8291901458672 | Validation Precision: 74.65597164051846 | Validation Recall: 73.24840764331209
Ea

<All keys matched successfully>

In [65]:

# Test verisi üzerinde modelin doğruluğunu hesaplayalım
test_acc, test_f1, test_precision, test_recall = evaluate(model_dropout, test_dataloader)
print('Test Accuracy:', test_acc, '| Test F1:', test_f1, '| Test Precision:', test_precision, '| Test Recall:', test_recall)

Test Accuracy: 67.09844559585493 | Test F1: 66.85631294439585 | Test Precision: 67.5535329451144 | Test Recall: 67.09844559585493


In [66]:
for epoch in range(10):
    train(model, train_dataloader, optimizer, loss_fn)
    acc = evaluate(model, val_dataloader)
    print('Epoch:', epoch+1, '| Validation Accuracy:', acc)

Epoch: 1 | Validation Accuracy: (92.35668789808918, 92.35668789808918, 92.36418194936297, 92.35668789808918)
Epoch: 2 | Validation Accuracy: (92.67515923566879, 92.67047654651385, 92.81844219133151, 92.67515923566879)
Epoch: 3 | Validation Accuracy: (92.67515923566879, 92.67493634094338, 92.6765936166171, 92.67515923566879)
Epoch: 4 | Validation Accuracy: (92.67515923566879, 92.66452250320573, 92.9733565358428, 92.67515923566879)
Epoch: 5 | Validation Accuracy: (92.99363057324841, 92.99306205901385, 93.0226859128133, 92.99363057324841)
Epoch: 6 | Validation Accuracy: (93.31210191082803, 93.31108434795121, 93.35741681515216, 93.31210191082803)
Epoch: 7 | Validation Accuracy: (92.99363057324841, 92.99192482293151, 93.05819396332303, 92.99363057324841)
Epoch: 8 | Validation Accuracy: (93.31210191082803, 93.31189841476566, 93.32873297641801, 93.31210191082803)
Epoch: 9 | Validation Accuracy: (92.67515923566879, 92.67404458114093, 92.68985910620134, 92.67515923566879)
Epoch: 10 | Validation

In [32]:
test_acc = evaluate(model, test_dataloader)
print('Test Accuracy:', test_acc)

Test Accuracy: 88.08290155440415
